<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 作業用ディレクトリへ移動
# システムコマンドは基本「！」をつけるが、ディレクトリ移動は「％」らしい
!mkdir chap7
%cd ./chap7


/content/chap7


In [2]:
# いつものライブラリに加えて、ファインチューニングと性能評価を行うPyTorch Lightningも使用
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# またNo module named ‘torchtext.legacy’エラーが出たため
# https://masaki-note.com/2022/05/27/torchtext_legacy_error/
# に従い以下を実行
# !pip install --upgrade torchtext==0.9.1

# いろいろなライブラリたち
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 13.8 MB/s 
     |████████████████████████████████| 486 kB 62.9 MB/s 
     |████████████████████████████████| 13.4 MB 60.5 MB/s 
     |████████████████████████████████| 582 kB 63.2 MB/s 
     |████████████████████████████████| 120 kB 55.7 MB/s 
     |████████████████████████████████| 880 kB 59.0 MB/s 
     |████████████████████████████████| 6.6 MB 50.4 MB/s 
     |████████████████████████████████| 419 kB 61.7 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=4aab640787b6d5bf43e17b075378cb982dd13828c1cc03b1b0bdc7e76e2aa556
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1f056baedd56580015fe5f17e45cffe8c2e553aded801cf5fa90d94928a8fe3e
  Stored in dire

In [2]:
'''
マルチラベル分類：一つの文章に対して、複数のカテゴリを割り当てる分類方法
シングルラベル分類：一つの文章に対して、一つのカテゴリを割り当てる分類方法


## マルチラベル分類
文章の属すカテゴリーをMulti-hotベクトルと呼ぶ形式で表現する
  サイズがカテゴリ数と同じ
  文章が属しているカテゴリに対応する要素に１、それ以外に０が入る

  文章が属しているカテゴリ　＝　分類スコアが正の値のカテゴリ
  文章が属していないカテゴリ　＝　分類スコアが負の値のカテゴリ
  とする（０はどっちに入るのか分からない・・・）


## BertForSequenceClassificationMultiLabel
マルチラベル分類を行うクラス（あとで定義します）

入力：文章（複数バッチ、符号化済み）
出力：文章ごとのMulti-hotベクトル


処理
１。入力をBertModelにつっこみ、出力（各トークンに対する分散表現）を得る
２。得られたベクトルを[PAD]（系列長長さ合わせのために挿入されたトークン）に対応するものを除いて平均化する
３。平均化したものを線形変換する
４。上記で得られた入力の１文ごとに対するMulti-hotベクトルが出力

BertForSequenceClassification(シングルラベル分類)との違い
  損失関数がバイナリクロスエントロピー（シングルの時はただのクロスエントロピーだった）
    cf : https://yaakublog.com/crossentropy_binarycrossentropy
    
    分類スコアに対してシグモイド関数を適用した値を計算する
      シグモイド関数の出力：そのカテゴリに文章が属する確率→50%以上の時を正とみなす
    そして、そうやって求めた予測確立と実際に文章がそのカテゴリに属しているかどうかとの間のバイナリクロスエントロピーを求める

  

'''